In [1]:
# load and evaluate a saved model
from numpy import loadtxt
from keras.models import load_model
import pickle
import pandas as pd 
import numpy as np
import time 

Using TensorFlow backend.


In [2]:
#load model
model = load_model("./models/mlp_dae.h5")
# summarize model.
model.summary()
with open("./models/LabelEncoders_dic.pickle","rb") as f:
    encoder_dic=pickle.load(f)
with open("./models/MinMaxScalers_dic.pickle","rb") as f:
    scaler_dic=pickle.load(f)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 115)          0                                            
__________________________________________________________________________________________________
dense_11 (Dense)                (None, 115)          13340       input_2[0][0]                    
__________________________________________________________________________________________________
dense_12 (Dense)                (None, 230)          26680       dense_11[0][0]                   
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 230)          0           dense_12[0][0]                   
____________________________________________________________________________________________

In [3]:
def transformer_df(df):
    for c in df.columns :
        if (df[c].dtype =="object"):
            encoder = encoder_dic[c]
            df[c]=encoder.transform(df[c])
    return df

In [4]:
def normaliser_all_columns(df):
    for c in df.columns :
        df[c]=scaler_dic[c].transform(df[c].values.reshape(-1,1))
    return df

In [6]:
df =pd.read_csv("../../../data/full_data_small_datased.csv")

In [7]:
test_df = pd.DataFrame()
test_df = test_df.append(df.iloc[0])
test_df.head()

,Class,HH_L0.01_covariance,HH_L0.01_magnitude,HH_L0.01_mean,HH_L0.01_pcc,HH_L0.01_radius,HH_L0.01_std,HH_L0.01_weight,HH_L0.1_covariance,HH_L0.1_magnitude,...,MI_dir_L0.1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L1_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L3_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L5_weight
0,0.0,0.0,60.0,60.0,0.0,0.0,0.0,1.0,0.0,60.0,...,1.0,60.0,0.0,1.0,60.0,0.0,1.0,60.0,0.0,1.0


In [8]:
target = test_df.Class

In [9]:
x_test = test_df.drop(['Class'],axis=1)

In [10]:
x_test=transformer_df(x_test)
x_test=normaliser_all_columns(x_test)

In [15]:
y_pred=np.round(model.predict([x_test,x_test])) 

In [20]:
y_pred[0]

array([0.], dtype=float32)